> First time use: follow instructions in the README.md file in this directory.



# Receitas

Várias receitas para usar Timelink com notebooks

[Sumário]

---

# Receipts

Receipts for using Timelink with notebooks

[summary]


---
## Setup

### Basic setup

Create a `TimelinkNotebook` instance.

This will provide an interface to the 
various `Timelink` functions.



In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()


### Changing the default values

When called with no parameters `TimelinkNotebook` 
assumes a certain directory layout containing the notebook
and from that layout autoconfigures various parameters.

The default layout:

* **project-directory**
    * **database** (database related filed)
    * **notebooks**  _(directory with the current notebook)_ 
    * **kleio**  (source files in kleio format)
    * **identifications** (files with identification information)

Based on this layout `TimelinkNotebook` assumes a few default values:

* **database name**
    * the name of the *project directory* is used as the **database name**, sanitized to produce a valid database name
      (hifens and spaces are replaced with _ ).
* **database type**
  * the type of the database is assumed to be **sqlite**  and a **sqlite** database is created in 
      *database/sqlite*
    * the `db_type` parameter can be used to set the database to `postgres`. 
* **Kleio server home**, working directory for the _Kleio server_ is assumed to be the **project directory**
  * The _Kleio server_ is an application
    that processes transcriptions of historical _Kleio_ notation and generates data in a format
    that can be imported to the database.

These default values can be changed with parameters when creating the `TimelinkNotebook`  instance:

```python
tlnb = TimelinkNotebook(
    project_name="my-project",
    project_home="~/projects/m-project",
    db_type="postgres",
    db_name="my_project_db",
    kleio_image="timelinkserver/kleio-server",
    kleio_version="12.0.0",
    postgres_image="postgres",
    postgres_version="latest",
    sqlite_dir="~/databases",
    sql_echo=True
)
```

### Examining the current configuration

In [2]:

tlnb.print_info()


Project name: dehergne-locations
Project home: /Users/jrc/mhk-home/sources/dehergne-locations
Database type: sqlite
Database name: dehergne_locations
Kleio image: timelinkserver/kleio-server
Kleio server token: vL8S5ktmqWOksA3eglKxYT86UFoy2xRj
Kleio server URL: http://127.0.0.1:8089
Kleio server home: /Users/jrc/mhk-home/sources/dehergne-locations
Kleio server container: awesome_sanderson
Kleio version requested: latest
Kleio server version: 12.4.567 (2024-02-07 13:02:03)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne-locations/database/sqlite
TimelinkNotebook(project_name=dehergne-locations, project_home=/Users/jrc/mhk-home/sources/dehergne-locations, db_type=sqlite, db_name=dehergne_locations, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


### Listing available databases

In [3]:
pd = tlnb.get_postgres_databases()
print(f"Postgres databases: {pd}")
sd = tlnb.get_sqlite_databases()
print(f"Sqlite databases: {sd}")


Postgres databases: ['timelink']
Sqlite databases: ['../database/sqlite/dehergne_locations.sqlite']


### Database status (table row count)

In [4]:
tlnb.table_row_count_df()

,table,count
0,acts,29
1,aregisters,1
2,attributes,25482
3,class_attributes,70
4,classes,14
5,entities,31624
6,geoentities,359
7,goods,0
8,kleiofiles,31
9,objects,0


## Dealing with kleio files

### List Kleio files available

In [5]:
kleio_files = tlnb.get_kleio_files()
kleio_files.head()

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,sources/dehergne-a.cli,dehergne-a.cli,2023-11-29 02:16:33.838462+00:00,V,2023-10-13 04:46:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-07 03:22:59.712358,/rest/reports/sources/dehergne-a.rpt,/rest/exports/sources/dehergne-a.xml
1,sources/dehergne-b.cli,dehergne-b.cli,2023-11-29 02:16:33.847301+00:00,V,2023-10-13 04:47:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-07 03:23:11.907971,/rest/reports/sources/dehergne-b.rpt,/rest/exports/sources/dehergne-b.xml
2,sources/dehergne-c.cli,dehergne-c.cli,2023-11-29 02:16:33.858180+00:00,V,2023-10-13 04:47:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-07 03:23:29.418688,/rest/reports/sources/dehergne-c.rpt,/rest/exports/sources/dehergne-c.xml
3,sources/dehergne-d.cli,dehergne-d.cli,2023-11-29 02:16:33.873745+00:00,V,2023-10-13 04:47:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-07 03:23:36.819380,/rest/reports/sources/dehergne-d.rpt,/rest/exports/sources/dehergne-d.xml
4,sources/dehergne-e.cli,dehergne-e.cli,2023-11-29 02:16:33.879442+00:00,V,2023-10-13 04:47:00+00:00,0,0,I,0,0,No errors,No warnings,2024-02-07 03:23:38.526859,/rest/reports/sources/dehergne-e.rpt,/rest/exports/sources/dehergne-e.xml


Show only translation and import status

In [6]:
kleio_files[["name","status","import_status"]]

,name,status,import_status
0,dehergne-a.cli,V,I
1,dehergne-b.cli,V,I
2,dehergne-c.cli,V,I
3,dehergne-d.cli,V,I
4,dehergne-e.cli,V,I
5,dehergne-f.cli,V,I
6,dehergne-g.cli,V,I
7,dehergne-h.cli,V,I
8,dehergne-i.cli,V,I
9,dehergne-j.cli,V,I


Show translation and import errors

In [7]:
kleio_files[["name","errors","warnings","import_errors","import_warnings"]]

,name,errors,warnings,import_errors,import_warnings
0,dehergne-a.cli,0,0,0,0
1,dehergne-b.cli,0,0,0,0
2,dehergne-c.cli,0,0,0,0
3,dehergne-d.cli,0,0,0,0
4,dehergne-e.cli,0,0,0,0
5,dehergne-f.cli,0,0,0,0
6,dehergne-g.cli,0,0,0,0
7,dehergne-h.cli,0,0,0,0
8,dehergne-i.cli,0,0,0,0
9,dehergne-j.cli,0,0,0,0


### Translation and Import reports

Translation reports.

Pass a dataframe with Kleio files and a row number to get the translation report

In [10]:
rpt=tlnb.get_translation_report(kleio_files,rows=6)
print(rpt)

KleioTranslator - server version 12.0 - build 541 (MHK: 2235/master) (2023-07-04 04:34:23, MHK: 06/08/2023 04:15 )
13-10-2023 4-47

Processing data file dehergne-g.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 116
Obs: 
** Processing source fonte$dehergne-g
7: lista$dehergne-notices-g
*** End of File


Structure file: /kleio-home/sources/dehergne/structures/sources.str
Structure processing report: /kleio-home/sources/dehergne/structures/sources.srpt
Structure in JSON: /kleio-home/sources/dehergne/structures/sources.str.json

Kleio file: /kleio-home/sources/dehergne/sources/dehergne-g.cli
Original file: /kleio-home/sources/dehergne/sources/dehergne-g.org
Previous version: /kleio-home/sources/dehergne/sources/dehergne-g.old
Temp file with ids: /kleio-home/sources/dehergne/sourc

Or use the file name

In [11]:
file = kleio_files.iloc[6].rpt_url
print(file)
rpt=tlnb.get_translation_report(file)
print(rpt)

/rest/reports/sources/dehergne-g.rpt
KleioTranslator - server version 12.0 - build 541 (MHK: 2235/master) (2023-07-04 04:34:23, MHK: 06/08/2023 04:15 )
13-10-2023 4-47

Processing data file dehergne-g.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 116
Obs: 
** Processing source fonte$dehergne-g
7: lista$dehergne-notices-g
*** End of File


Structure file: /kleio-home/sources/dehergne/structures/sources.str
Structure processing report: /kleio-home/sources/dehergne/structures/sources.srpt
Structure in JSON: /kleio-home/sources/dehergne/structures/sources.str.json

Kleio file: /kleio-home/sources/dehergne/sources/dehergne-g.cli
Original file: /kleio-home/sources/dehergne/sources/dehergne-g.org
Previous version: /kleio-home/sources/dehergne/sources/dehergne-g.old
Temp file with id

Import report

In [12]:
rpt = tlnb.get_import_rpt(kleio_files,rows=6)
print(rpt)

No errors
No warnings



# Atualizar base de dados

Atualiza traduções de fontes e importa quando tradução não tem erros.

---

# Update database

Updates source translations and imports into database sources with no errors.

In [47]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()

Show imported files status

In [48]:
imported_files_df = tlnb.get_import_status()
imported_files_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   path                30 non-null     object             
 1   name                30 non-null     object             
 2   size                30 non-null     int64              
 3   directory           30 non-null     object             
 4   modified            30 non-null     datetime64[ns, UTC]
 5   modified_iso        30 non-null     datetime64[ns, UTC]
 6   modified_string     30 non-null     object             
 7   qtime               30 non-null     datetime64[ns, UTC]
 8   qtime_string        30 non-null     object             
 9   source_url          30 non-null     object             
 10  status              30 non-null     object             
 11  translated          30 non-null     datetime64[ns, UTC]
 12  translated_string   30 non-null     ob

Check the import status of the translated files
```python
    I = "I" # imported
    E = "E" # imported with error
    W = "W" # imported with warnings no errors
    N = "N" # not imported
    U = "U" # translation updated need to reimport

``` 

In [50]:
imported_files_df[["import_status","import_errors","import_warnings","name","imported","path"]].sort_values("name")

,import_status,import_errors,import_warnings,name,imported,path
0,I,0,0,dehergne-a.cli,2024-02-07 03:22:59.712358,sources/dehergne-a.cli
1,I,0,0,dehergne-b.cli,2024-02-07 03:23:11.907971,sources/dehergne-b.cli
2,I,0,0,dehergne-c.cli,2024-02-07 03:23:29.418688,sources/dehergne-c.cli
3,I,0,0,dehergne-d.cli,2024-02-07 03:23:36.819380,sources/dehergne-d.cli
4,I,0,0,dehergne-e.cli,2024-02-07 03:23:38.526859,sources/dehergne-e.cli
5,I,0,0,dehergne-f.cli,2024-02-07 03:23:49.669086,sources/dehergne-f.cli
6,I,0,0,dehergne-g.cli,2024-02-07 03:24:00.428726,sources/dehergne-g.cli
7,I,0,0,dehergne-h.cli,2024-02-07 03:24:04.137667,sources/dehergne-h.cli
8,I,0,0,dehergne-i.cli,2024-02-07 03:24:06.156498,sources/dehergne-i.cli
9,I,0,0,dehergne-j.cli,2024-02-07 03:24:08.424135,sources/dehergne-j.cli


## Todo

This as data frame in a single function
* TimelinkNotebook.translate([files_df,rows=List[int],status="T"])
* TimelinkNotebook.import(files_df, rows=List[int])



# Obter dados
---

# Getting data

### Procurar pessoa

---
### Search for people

#### Search persons by name

In [52]:
from timelink.api.models import Person
from sqlalchemy import select

show_only=10

with tlnb.db.session() as session:
    stmt = select(Person).where(Person.name.like('% ruggieri'))
    print(stmt)
    persons = session.execute(stmt).scalars().all()
    print()
    for person in persons[:show_only]:
        print(person.id, person.name,person.sex)

SELECT persons.id, entities.id AS id_1, entities.class, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, persons.name, persons.sex, persons.obs 
FROM entities JOIN persons ON entities.id = persons.id 
WHERE persons.name LIKE :name_1

deh-baltazar-y-gonzalo-ref1 Michele Ruggieri m
deh-bernardino-ferrario-ref1 Michele Ruggieri m
deh-pedro-gomez-ref1 Michele Ruggieri m
deh-andre-pinto-ref4 Michele Ruggieri m
deh-francisco-pires-ref1 Michele Ruggieri m


### Search other Entities

#### get the Entity classes in the database

In [53]:
from sqlalchemy import select, func
from timelink.api.models import Entity

models = tlnb.db.get_models_ids()
models

['attribute',
 'relation',
 'act',
 'source',
 'person',
 'good',
 'object',
 'class',
 'geoentity',
 'rgeoentity',
 'robject',
 'rperson',
 'rentity',
 'aregister',
 'entity']

#### Get columns of an entity type

In [54]:
table = tlnb.db.get_table("entity")
print(table.name)
list(table.columns)

entities


[Column('id', String(), table=<entities>, primary_key=True, nullable=False),
 Column('class', String(), table=<entities>, nullable=False),
 Column('inside', String(), ForeignKey('entities.id'), table=<entities>),
 Column('the_order', Integer(), table=<entities>),
 Column('the_level', Integer(), table=<entities>),
 Column('the_line', Integer(), table=<entities>),
 Column('groupname', String(), table=<entities>),
 Column('updated', DateTime(), table=<entities>, default=CallableColumnDefault(<function datetime.utcnow at 0x133fc00d0>)),
 Column('indexed', DateTime(), table=<entities>)]

#### Search any entity type

## IMPROVE

In [56]:
from timelink.api.models import Entity
from sqlalchemy import select, func

Geoentity = tlnb.db.get_model("geoentity")
stmt = select(Geoentity).where(Geoentity.name.like('Hang%'))
print(stmt)
with tlnb.db.session() as session:

    result = session.execute(stmt).scalars().all()
    for row in result:
        print()
        print(row.the_type,row.name,row.id,row.obs)
        print(row.to_kleio())

SELECT geoentities.id, entities.id AS id_1, entities.class, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, geoentities.name, geoentities.obs, geoentities.the_type 
FROM entities JOIN geoentities ON entities.id = geoentities.id 
WHERE geoentities.name LIKE :name_1

geo2 Hangchou deh-r1644-hangchou None
geo2$deh-r1644-hangchou/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q4970"/16440000
  atr$activa/sim/16110000
  atr$residencia-missao/Jesuíta/16110000
  geo3$deh-r1644-fuyang/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q1011103"/16440000
    atr$activa/sim/16420000
  geo3$deh-r1644-jenho/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/16080000

geo2 Hangchou deh-r1701-hangchou None
geo2$deh-r1701-hangch

#### Get person by id

Show a single person or entity in Kleio notation

In [58]:
p = tlnb.db.get_person('deh-matteo-ricci')
print(p.to_kleio())

n$Matteo Ricci/m/id=deh-matteo-ricci
  rel$function-in-act/n/dehergne-notices-r/0
  ls$nacionalidade/Itália/0
  ls$jesuita-estatuto/Padre/0
  ls$nome-chines/Li Ma-teou/0
  ls$nome-chines/Si-T'ai/0
  ls$nascimento@/"https://www.wikidata.org/wiki/Q13135"/15521006
  ls$nascimento/Macerata/15521006
  ls$jesuita-entrada@/"https://www.wikidata.org/wiki/Q220"/15710816
  ls$jesuita-entrada/Roma (Noviciado de Sant'Andrea al Quirinale)/15710816
  ls$embarque/S. Luís/15780324
  ls$wicky/200/15780324
  ls$wicky-viagem/25/15780324
  ls$chegada@/"https://www.wikidata.org/wiki/Q1171"/15780913
  ls$chegada/Goa/15780913
  ls$estadia@/"https://www.wikidata.org/wiki/Q1800"/15800726
  ls$estadia/Cochim/15800726
  ls$tarefa/Celebra a primeira missa/15800726
  ls$chegada@/"https://www.wikidata.org/wiki/Q14773"/15820807
  ls$chegada/Macau/15820807
  ls$estadia@/"https://www.wikidata.org/wiki/Q16572"/15830800
  ls$estadia/Cantão/15830800
  ls$estadia/Shiuhing (Tchao-k'ing fou)/15830910
  ls$jesuita-tarefa/Fun

### Show other type of entities by id in Kleio

In [59]:
ent = tlnb.db.get_entity("deh-r1644-chekiang")
print(ent.to_kleio())

geo1$deh-r1644-chekiang/type=geoentity
  rel$function-in-act/geo1/deh-chre-1644/16440000
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q16967"/16440000
  geo2$deh-r1644-hangchou/type=geoentity
    rel$function-in-act/geo2/deh-chre-1644/16440000
    atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q4970"/16440000
    atr$activa/sim/16110000
    atr$residencia-missao/Jesuíta/16110000
    geo3$deh-r1644-fuyang/type=geoentity
      rel$function-in-act/geo3/deh-chre-1644/16440000
      atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q1011103"/16440000
      atr$activa/sim/16420000
    geo3$deh-r1644-jenho/type=geoentity
      rel$function-in-act/geo3/deh-chre-1644/16440000
      atr$activa/sim/16080000
  geo2$deh-r1644-chuchow/type=geoentity
    rel$function-in-act/geo2/deh-chre-1644/16440000
    atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q58235"/16440000
    atr$activa/sim/16130000
  geo2$deh-r1644-huchow/type=geoentity
    rel$function-i



###  Obter um dataframe a partir de atributos

---


###  Get a Dataframe from attributes


#### Exemplo: Faculdade, data de entrada e data de saída e grau dos naturais de Coimbra

In [69]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='person',
                    the_type='jesuita-entrada',
                    the_value='Coimbra',
                    more_info=['name','sex'],
                    name_like='%',
                    more_cols=['nascimento','morte'],
                    db=tlnb.db,
                    sql_echo=False)
conimbricensis = df.index.values
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43 entries, deh-pedro-de-alcacova to deh-inacio-pires
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  43 non-null     object
 1   sex                   43 non-null     object
 2   jesuita-entrada       43 non-null     object
 3   jesuita-entrada.date  43 non-null     object
 4   jesuita-entrada.obs   1 non-null      object
 5   nascimento            43 non-null     object
 6   nascimento.date       43 non-null     object
 7   nascimento.obs        0 non-null      object
 8   morte                 36 non-null     object
 9   morte.date            36 non-null     object
 10  morte.obs             2 non-null      object
dtypes: object(11)
memory usage: 5.1+ KB


In [64]:
df

,name,sex,jesuita-entrada,jesuita-entrada.date,jesuita-entrada.obs,nascimento,nascimento.date,nascimento.obs,morte,morte.date,morte.obs
id,,,,,,,,,,,
deh-pedro-de-alcacova,Pedro de Alcáçova,m,Coimbra,15420000,saiu a primeira vez,?,15230000,None,?,15790000,None
deh-belchior-nunes-barreto,Belchior Nunes Barreto,m,Coimbra,15430311,None,Porto,15190000,None,Goa,15710810,None
deh-belchior-miguel-carneiro-leitao,Belchior Miguel Carneiro Leitão,m,Coimbra,15430425,None,Coimbra,15190000,None,Macau,15830819,None
deh-francisco-perez,Francisco Pérez,m,Coimbra,15440125,None,"Villanueva de Barcarrota, diocese de Badajoz",15140000,None,Negapatam,15830222,None
deh-tiburcio-de-quadros,Tibúrcio de Quadros,m,Coimbra,15440418,None,Santarém,15280000,None,Goa,15721121,None
deh-goncalo-alvares,Gonçalo Álvares,m,Coimbra,15490101,None,"Vila Viçosa, diocese de Évora",15270000,None,[A caminho do Japão],15730700,morre afogado com quatro companheiros no camin...
deh-cristovao-da-costa,Cristóvão da Costa,m,Coimbra,15500103,None,?,15290000,None,Macau,15820000,None
deh-joao-rodrigues-de-sainan,João Rodrigues de Sainan,m,Coimbra,15550000,None,?,15310000,None,NaN,NaN,NaN
deh-pedro-martins,Pedro Martins,m,Coimbra,15560525,None,Coimbra,15420000,None,"[No mar, a caminho de Malaca]",15980213,None


#### obter attributos de outras entidades

In [73]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='geoentity',
                    more_info=['name'],
                    the_type='activa',
                    the_value='sim',
                    more_cols=['residencia-missao'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253 entries, deh-r1644-anhai to deh-r1701-yunnanfu
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    253 non-null    object
 1   activa                  253 non-null    object
 2   activa.date             253 non-null    object
 3   activa.obs              5 non-null      object
 4   residencia-missao       88 non-null     object
 5   residencia-missao.date  88 non-null     object
 6   residencia-missao.obs   11 non-null     object
dtypes: object(7)
memory usage: 15.8+ KB


In [75]:
df.head(30)

,name,activa,activa.date,activa.obs,residencia-missao,residencia-missao.date,residencia-missao.obs
id,,,,,,,
deh-r1644-anhai,Anhai,sim,1634,None,NaN,NaN,NaN
deh-r1644-bankao,Bankao,sim,1636,None,NaN,NaN,NaN
deh-r1644-cantao,Cantão,sim,1555,None,Jesuíta,1580,None
deh-r1644-cantao,Cantão,sim,1555,None,Franciscanos,0000,None
deh-r1644-cantao,Cantão,sim,1555,None,Dominicanos,0000,None
deh-r1644-chala,Chala,sim,1610,None,NaN,NaN,NaN
deh-r1644-changchow-fou,Changchow,sim,1643,None,NaN,NaN,NaN
deh-r1644-changshu,Changshu,sim,1623,None,Jesuíta,1635,"R 1635 «Cham Xo», cf AHSI 28 (1951) 311-312"
deh-r1644-chengting,Chengting,sim,1621,None,NaN,NaN,NaN




###  Remover colunas sem valores

---



###  Remove empty columns

In [76]:
df.dropna(how='all', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253 entries, deh-r1644-anhai to deh-r1701-yunnanfu
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    253 non-null    object
 1   activa                  253 non-null    object
 2   activa.date             253 non-null    object
 3   activa.obs              5 non-null      object
 4   residencia-missao       88 non-null     object
 5   residencia-missao.date  88 non-null     object
 6   residencia-missao.obs   11 non-null     object
dtypes: object(7)
memory usage: 15.8+ KB


In [77]:
df.head(5)

,name,activa,activa.date,activa.obs,residencia-missao,residencia-missao.date,residencia-missao.obs
id,,,,,,,
deh-r1644-anhai,Anhai,sim,1634,None,NaN,NaN,NaN
deh-r1644-bankao,Bankao,sim,1636,None,NaN,NaN,NaN
deh-r1644-cantao,Cantão,sim,1555,None,Jesuíta,1580,None
deh-r1644-cantao,Cantão,sim,1555,None,Franciscanos,0000,None
deh-r1644-cantao,Cantão,sim,1555,None,Dominicanos,0000,None


In [78]:
ids = df.index.unique()
for id in ids[:5]:
    ent = tlnb.db.get_entity(id)
    print(ent.to_kleio())
    print()

geo3$deh-r1644-anhai/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  atr$geoentity:type@wikidata/"https://www.wikidata.org/wiki/Q4764330"/16440000
  atr$activa/sim/1634

geo3$deh-r1644-bankao/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  atr$activa/sim/1636

geo2$deh-r1644-cantao/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q16572"/16440000
  atr$activa/sim/1555
  atr$residencia-missao/Jesuíta/1580
  atr$residencia-missao/Franciscanos/0000
  atr$residencia-missao/Dominicanos/0000
  geo3$deh-r1644-tungkun/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q59218"/16440000
  geo3$deh-r1644-quon-yao/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/1621
  geo3$deh-r1644-lampacao/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$geoe


## Contagens

---

## Counting



###  Contagem de atributos a partir de uma tabela em memória

---

###  Count attributes from an existing dataframe



In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253 entries, deh-r1644-anhai to deh-r1701-yunnanfu
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    253 non-null    object
 1   activa                  253 non-null    object
 2   activa.date             253 non-null    object
 3   activa.obs              5 non-null      object
 4   residencia-missao       88 non-null     object
 5   residencia-missao.date  88 non-null     object
 6   residencia-missao.obs   11 non-null     object
dtypes: object(7)
memory usage: 23.9+ KB


In [80]:
# create a column with the index values which are the id numbers
df['id'] = df.index.values

col = 'residencia-missao' # subotal by this column

# Use pandas groupby and specify unique value count for id
df_totals = df.groupby(col).agg({'residencia-missao.date':['count','min','max']}

                                )
df_totals#.sort_values('id',ascending= False).head(30)

residencia-missao.date                    
                                      count       min       max
residencia-missao                                              
Agostinhos                                5      1701      1701
Dominicanos                              15      0000      1701
Franciscanos                             21      0000      1701
Jesuíta                                  37      1580      1701
Lazaristas                                1  17020400  17020400
Missões Estrangeiras                      2  17020400  17021100
Missões estrangeiras                      6      1701      1701
Propaganda                                1      1703      1703


### Contagens na base de dados

Quando o atributo tem muitos valores e não é necessário
ter todas as pessoas em memória: contagem feita na base de dados

---

### Counting directly in the database
When there are many values and it is not
necessary to have all the people in memory:
count directly in the database.




In [84]:
from timelink.pandas import attribute_values

df_totals = attribute_values('estadia',db=tlnb.db)


In [85]:
df_totals.head(10)


,count,date_min,date_max
value,,,
Macau,300,0,1773
Pequim,189,0,17910000
Cantão,157,0,17850000
Goa,116,0,17860123
China,79,0,17680000
Lisboa,74,0,17810300
Nanquim,52,0,17800000
Roma,50,0,17750000
Tonquim,44,0,17800000


##ERROR

## Visualizar registos

---

## View records





### Visualizar uma pessoa

---

### View a person


#### Atributos de uma pessoa numa tabela, uma linha por attributo

---

#### Person attributes in a dataframe, one line per attribute

In [87]:
import pandas as pd
from timelink.pandas import group_attributes as person_attributes

pd.set_option('display.max_rows',1000)

id = 'deh-matteo-ricci'
pdf = person_attributes([id],db=tlnb.db)  # note id in a list
pdf[['date','type','value','attr_obs']].sort_values(['date','type'])

,date,type,value,attr_obs
id,,,,
deh-matteo-ricci,0,dehergne,684,"Ricci, Matteo (ital.) P. 684 Li Ma-teou, Si-T'..."
deh-matteo-ricci,0,dehergne,684,"P. 25 juill. 1580, à Cochin, où il célèbre sa ..."
deh-matteo-ricci,0,dehergne,684,"D'Elu III, 239-243. Biographie: en chinois: BN..."
deh-matteo-ricci,0,dehergne,684,"Tacchi-Venturi, Opère storiche del P. M. R., M..."
deh-matteo-ricci,0,dehergne,684,id.. L'apport scientifique du P. M. R à la Chi...
deh-matteo-ricci,0,dehergne,684,"Bettray, Johannes, S.V.D., Die akkomodationmet..."
deh-matteo-ricci,0,dehergne,684,"Sur l'art chinois: Sepp Schûller, P. M. Ricci ..."
deh-matteo-ricci,0,dehergne,684,"— Délia Entrata delta Comp. nella Cina, avec a..."
deh-matteo-ricci,0,dehergne,684,"pécimen d'écriture, nom chinois et romanisatio..."


#### Atributos de uma pessoa numa tabela, attributos em colunas

---

#### Person attributes in a dataframe, attributes in columns

In [96]:
# Get list of people with with a certain value in a specific attribute

df = entities_with_attribute(
                    entity_type='person',
                    the_type='jesuita-entrada',  # we need a base attribute
                    more_info=['name','sex'],
                    more_cols=['jesuita-entrada.date','nascimento','chegada','morte'],
                    filter_by=conimbricensis,
                    db=tlnb.db,
                    sql_echo=False)
view_cols = ['name','nascimento','nascimento.date','jesuita-entrada','jesuita-entrada.date','chegada','morte']
df[view_cols].sort_values('nascimento.date')

,name,nascimento,nascimento.date,jesuita-entrada,jesuita-entrada.date,chegada,morte
id,,,,,,,
deh-joao-rodrigues-girao,João Rodrigues Girão,"Alcochete, diocese de Lisboa",0,Coimbra,None,Japão,?
deh-francisco-perez,Francisco Pérez,"Villanueva de Barcarrota, diocese de Badajoz",15140000,Coimbra,None,NaN,Negapatam
deh-belchior-miguel-carneiro-leitao,Belchior Miguel Carneiro Leitão,Coimbra,15190000,Coimbra,None,Macau,Macau
deh-belchior-nunes-barreto,Belchior Nunes Barreto,Porto,15190000,Coimbra,None,Japão,Goa
deh-belchior-miguel-carneiro-leitao,Belchior Miguel Carneiro Leitão,Coimbra,15190000,Coimbra,None,Goa,Macau
deh-pedro-de-alcacova,Pedro de Alcáçova,?,15230000,Goa,None,NaN,?
deh-pedro-de-alcacova,Pedro de Alcáçova,?,15230000,Coimbra,None,NaN,?
deh-goncalo-alvares,Gonçalo Álvares,"Vila Viçosa, diocese de Évora",15270000,Coimbra,None,NaN,[A caminho do Japão]
deh-tiburcio-de-quadros,Tibúrcio de Quadros,Santarém,15280000,Coimbra,None,NaN,Goa


#### Examinar potenciais duplicados

---

#### Examine potentital duplicates

In [15]:
from timelink.pandas import display_group_attributes
pd.set_option('display.max_rows',250)

no_show=['código-de-referência','data-do-registo','url','faculdade.ano','naturalidade.ano',
         'matricula-faculdade.ano','nome-apelido','nome-primeiro','nome-geografico.ano',
         'grau.ano','matricula-outra.ano','nome-geografico','instituta.ano']

dup_ids = ['234295','234710',]  # Alexandre Metelo de

display_group_attributes(dup_ids,
                             header_cols=['uc-entrada','naturalidade','faculdade','nome-pai'],
                             exclude_attributes=no_show,
                             sort_attributes=['date','type','value'],
                             cmap_name='Pastel1')

,id,uc-entrada,naturalidade,faculdade,nome-pai
0,234295,1704-11-07,Marialva,Cânones,nan
1,234710,1705-10-24,Marialva,Matemática,Manuel Cardoso Metelo


,date,id,type,value,attr_obs
0,1704-11-07,234295,faculdade,Cânones,Cânones
1,1704-11-07,234295,instituta,1704-11-07,07.11.1704 1704-11-07
2,1704-11-07,234295,naturalidade,Marialva,None
3,1704-11-07,234295,nome,Alexandre Metelo de Sousa,None
4,1704-11-07,234295,uc-entrada,1704-11-07,None
5,1704-11-07,234295,uc-entrada.ano,1704,None
6,1705-10-24,234710,faculdade,Matemática,Matemática
7,1705-10-24,234295,matricula-faculdade,Cânones,24.10.1705
8,1705-10-24,234710,matricula-faculdade,Matemática,24.10.1705
9,1705-10-24,234710,naturalidade,Marialva,None


#### Notação Kleio

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation

See [Kleio notation](README_kleio.md)

#### Notação Kleio directamente da base de dados

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation directly from database

See [Kleio notation](README_kleio.md)

In [33]:
from timelink.mhk.models.person import Person

with tlnb.db.session() as session:

    p: Person = session.query(Person).order_by(Person.id).first()
    k = p.to_kleio()
    print(p.to_kleio())


n$António Pinto Abadeço/m/id=140337/obs="""
      """

                  Id: 140337
                  Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/A/000001

                  Nome        : António Pinto Abadeço
                  Data inicial: 1705-10-01
                  Data final  : 1710-10-01
                  Filiação: António Pinto

                  Naturalidade: Abrantes
                  Faculdade: Cânones

                  Matrícula(s): 01.10.1705
                  01.10.1706
                  01.01.1707
                  01.10.1708
                  01.10.1709
                  01.10.1710
                  Instituta:
                  Bacharel: 28.06.1709
                  Formatura:
                  Licenciado:
                  Doutor:

                  Outras informações:
              """
  """
  rel$function-in-act/n/António Pinto Abadeço/auc-alumni-A-140337-140771/20200211
  atr$código-de-referência/""PT/AUC/ELU/UC-AUC/B/001-001/A/000001""/2021-05-17
  atr$data-